## Workflow: Selecting Functions with a Dictionary and Creating Decorators

In [34]:
# 1. Load the data

products = [
    ["SKU", "description", "cost", "list_price", "units_in_stock", "target_margin"],
    ['  20776', 'Budget Device\tMini\n1000-pack', '$284.43', '$406.563', '673', '42.9%'],
    ['  13508', 'Budget Widget\tExtra Large\n1000-pack', '$207.49', '$270.203', '461','30.2%'],
    ['   1261', 'Super Whatsit\tMini\nDozen', '$1,091.33', '$1,485.85', '368', '36.1%'],
    ['  49367', 'Super Gizmo\tExtra Large\nGross', '$1,977.36', '$2,589.87', '2,585', '31.0%'],
    ['  39655', 'Student Gadget\tMini\nGross', '$437.97', '$656.749', '4,452', '50.0%']
]
# products

In [35]:
# 3. Put the get_XX functions into a dictionary ...

def clean_spaces(field):
    """Removes leading and trailing spaces"""
    return field.strip()

def clean_whitespace(field):
    """Changes tabs and newlines to spaces"""
    table = field.maketrans('\t\n', '  ')
    contents = field.translate(table)
    return contents

def clean_commas(field):
    """Removes commas"""
    return field.replace(',', '')

def clean_currency(field):
    """Removes '$'"""
    return field.replace('$', '')

def clean_percent(field):
    """Removes '%'"""
    return field.replace('%', '')

#clean_spaces('  abc123  ')
#clean_whitespace('tabs\tand\nnewlines')
#clean_commas('1,234.00')
#clean_currency('$1,234.56')
#clean_percent('%100')

def get_sku(field):
    """returns the cleaned sku"""
    return clean_spaces(field)

def get_description(field):
    """returns the cleaned description"""
    return clean_whitespace(field)

def get_currency(field):
    """returns the cleaned currency field"""
    return float(clean_commas(clean_currency(field)))

def get_integer(field):
    """returns the cleaned integer field"""
    return int(clean_commas(field))

def get_percent(field):
    """returns the cleaned percentage as a fraction"""
    return float(clean_commas(clean_percent(field))) / 100

getters = { 0: get_sku, 1: get_description, 2: get_currency, 3: get_currency, 4: get_integer, 5: get_percent }
getters

#get_sku('  20776')
#get_description('Budget Device\tMini\n1000-pack')
#get_currency('$1,234.56')
#get_integer('2,585')
#get_percent('42.9%')

{0: <function __main__.get_sku(field)>,
 1: <function __main__.get_description(field)>,
 2: <function __main__.get_currency(field)>,
 3: <function __main__.get_currency(field)>,
 4: <function __main__.get_integer(field)>,
 5: <function __main__.get_percent(field)>}

In [36]:
# 4. loop over the rows and fields in the data... use the dictionary with the field number as the key

cleaned_products = []
cleaned_products.append(products[0])
for row in products[1:]:
    #print(row)
    clean_row = [getters[col](row[col]) for col in range(len(row))]
    #print(clean_row2)
    cleaned_products.append(clean_row)
cleaned_products

[['SKU',
  'description',
  'cost',
  'list_price',
  'units_in_stock',
  'target_margin'],
 ['20776', 'Budget Device Mini 1000-pack', 284.43, 406.563, 673, 0.429],
 ['13508', 'Budget Widget Extra Large 1000-pack', 207.49, 270.203, 461, 0.302],
 ['1261', 'Super Whatsit Mini Dozen', 1091.33, 1485.85, 368, 0.361],
 ['49367', 'Super Gizmo Extra Large Gross', 1977.36, 2589.87, 2585, 0.31],
 ['39655', 'Student Gadget Mini Gross', 437.97, 656.749, 4452, 0.5]]

In [37]:
# 5. create decorator functions from the specific cleaning functions...

from functools import wraps

def clean_spaces(func):
    @wraps(func)
    def wrapper(field):
        """Removes leading and trailing spaces"""
        field = field.strip()
        return func(field)
    return wrapper

def clean_whitespace(func):
    @wraps(func)
    def wrapper(field):
        """Changes tabs and newlines to spaces"""
        table = field.maketrans('\t\n', '  ')
        field = field.translate(table)
        return func(field)
    return wrapper

def clean_commas(func):
    @wraps(func)
    def wrapper(field):
        """Removes commas"""
        field = field.replace(',', '')
        return func(field)
    return wrapper

def clean_currency(func):
    @wraps(func)
    def wrapper(field):
        """Removes '$'"""
        field = field.replace('$', '')
        return func(field)
    return wrapper

def clean_percent(func):
    @wraps(func)
    def wrapper(field):
        """Removes '%'"""
        field = field.replace('%', '')
        return func(field)
    return wrapper

# test
@clean_spaces
def f1(field):
    print(field)
f1('  abc123  ')
@clean_whitespace
def f2(field):
    print(field)
f2('tabs\tand\nnewlines')
@clean_commas
def f3(field):
    print(field)
f3('1,234.00')
@clean_currency
def f4(field):
    print(field)
f4('$1,234.56')
@clean_percent
def f5(field):
    print(field)
f5('%100')

abc123
tabs and newlines
1234.00
1,234.56
100


In [38]:
# 6. Use the decorators with the field cleaning functions...

@clean_spaces
def get_sku(field):
    """returns the cleaned sku"""
    return field

@clean_whitespace
def get_description(field):
    """returns the cleaned description"""
    return field

@clean_currency
@clean_commas
def get_currency(field):
    """returns the cleaned currency field"""
    return float(field)

@clean_commas
def get_integer(field):
    """returns the cleaned integer field"""
    return int(field)

@clean_percent
@clean_commas
def get_percent(field):
    """returns the cleaned percentage as a fraction"""
    return float(field) / 100

getters = { 0: get_sku, 1: get_description, 2: get_currency, 3: get_currency, 4: get_integer, 5: get_percent }
#getters

# test
cleaned_products = []
cleaned_products.append(products[0])
for row in products[1:]:
    #print(row)
    clean_row = [getters[col](row[col]) for col in range(len(row))]
    #print(clean_row2)
    cleaned_products.append(clean_row)
cleaned_products

[['SKU',
  'description',
  'cost',
  'list_price',
  'units_in_stock',
  'target_margin'],
 ['20776', 'Budget Device Mini 1000-pack', 284.43, 406.563, 673, 0.429],
 ['13508', 'Budget Widget Extra Large 1000-pack', 207.49, 270.203, 461, 0.302],
 ['1261', 'Super Whatsit Mini Dozen', 1091.33, 1485.85, 368, 0.361],
 ['49367', 'Super Gizmo Extra Large Gross', 1977.36, 2589.87, 2585, 0.31],
 ['39655', 'Student Gadget Mini Gross', 437.97, 656.749, 4452, 0.5]]